# Chapter 8: Reissner-Mindlin plate

In [21]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
sp.init_printing()

### The physical properties of the element

In [22]:
E = 200e9  # Young's modulus in Pa
nu = 0.25  # Poisson's ratio
t = 1000  # thickness in mm
t /= 1000  # convert to m

# Define symbolic physical coordinates
x1, y1, x2, y2, x3, y3, x4, y4 = sp.symbols('x1 y1 x2 y2 x3 y3 x4 y4')
# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])

# Example values for the physical coordinates
# x1_val, y1_val = -1, -1
# x2_val, y2_val = 1, -1
# x3_val, y3_val = 1, 1
# x4_val, y4_val = -1, 1
#
# x1_val, y1_val = -1.1, -1.05
# x2_val, y2_val = 1.05, -1.05
# x3_val, y3_val = 1.05, 1.1
# x4_val, y4_val = -1.05, 1.1

x1_val, y1_val = 3, 2
x2_val, y2_val = 5, 2
x3_val, y3_val = 5, 4
x4_val, y4_val = 3, 4

# Create a dictionary to map the symbolic coordinates to their numerical values
coordinate_values = {
    x1: x1_val, y1: y1_val,
    x2: x2_val, y2: y2_val,
    x3: x3_val, y3: y3_val,
    x4: x4_val, y4: y4_val,
}

### The finite element

This is a rectangular isoparametric 2D plate in the x-y plane.

The natural coordinates are xi and eta. Node numbering is CCW, starting at ``(-1, -1)``:
```
     ^
     |eta
     |
  4--|--3
  |  |--|--->xi
  |     |
  1-----2
```

It has 3 degrees of freedom each node:
- w: vertical displacement
- thetaX: rotation about the x-axis
- thetaY: rotation about the y-axis
The DOFs are not coupled.
The DOFs are ordered as follows: w, thetaX, thetaY.


In [23]:
ND = 3  # number of degrees of freedom per node
NNODE = 4  # number of nodes

### The isoparametric part

In [24]:
n1 = np.array([-1, -1])  # node 1 coordinates
n2 = np.array([1, -1])   # node 2 coordinates
n3 = np.array([1, 1])    # node 3 coordinates
n4 = np.array([-1, 1])   # node 4 coordinates
natural_nodes = np.array([n1, n2, n3, n4])  # all nodes

### Shape functions and their derivatives

In [25]:
# Define symbolic variables
XI, ETA = sp.symbols('xi eta')

# Define the shape functions symbolically
def shape_functions() -> np.array:
    """shape functions"""

    return np.array([
        0.25 * (1 - XI) * (1 - ETA),
        0.25 * (1 + XI) * (1 - ETA),
        0.25 * (1 + XI) * (1 + ETA),
        0.25 * (1 - XI) * (1 + ETA),
    ])

def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    print(shape_functions())

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 1])])


# Calculate the derivatives by xi and eta, symbolically
def shape_function_derivatives() -> np.array:
    """shape function derivatives"""
    sh = shape_functions()
    dN_dxi = []
    dN_deta = []

    for i in range(len(sh)):
        dN_dxi.append(sp.diff(sh[i], XI))
        dN_deta.append(sp.diff(sh[i], ETA))

    return dN_dxi, dN_deta

print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i+1}/dxi = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i+1}/deta = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

print()
print('The derivatives of the shape functions by xi are:')
print(derivatives[0])  # derivatives by xi

print()
print('The derivatives of the shape functions by eta are:')
print(derivatives[1])  # derivatives by eta


The shape functions are:
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]

The derivatives are:
dN1/dxi = 0.25*eta - 0.25
dN2/dxi = 0.25 - 0.25*eta
dN3/dxi = 0.25*eta + 0.25
dN4/dxi = -0.25*eta - 0.25
dN1/deta = 0.25*xi - 0.25
dN2/deta = -0.25*xi - 0.25
dN3/deta = 0.25*xi + 0.25
dN4/deta = 0.25 - 0.25*xi

The isoparametric mapping is:
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]
x = x1*(0.25 - 0.25*xi)*(1 - eta) + x2*(1 - eta)*(0.25*xi + 0.25) + x3*(eta + 1)*(0.25*xi + 0.25) + x4*(0.25 - 0.25*xi)*(eta + 1)
y = y1*(0.25 - 0.25*xi)*(1 - eta) + y2*(1 - eta)*(0.25*xi + 0.25) + y3*(eta + 1)*(0.25*xi + 0.25) + y4*(0.25 - 0.25*xi)*(eta + 1)

The derivatives of the shape functions by xi are:
[0.25*eta - 0.25, 0.25 - 0.25*eta, 0.25*eta + 0.25, -0.25*eta - 0.2

### Example of the shape functions

In [26]:
# Evaluate the shape functions at xi=0, eta=0
shape_funcs_at = shape_functions()
print("Shape functions at xi=0, eta=0:")

for i, sf in enumerate(shape_funcs_at):
    print(f"N{i+1} = {sf.subs({XI: 0, ETA: 0})}")

print()
shape_funcs_at = shape_functions()
print("Shape functions at xi=1, eta=1:")
for i, sf in enumerate(shape_funcs_at):
    print(f"N{i+1} = {sf.subs({XI: 1, ETA: 1})}")


Shape functions at xi=0, eta=0:
N1 = 0.250000000000000
N2 = 0.250000000000000
N3 = 0.250000000000000
N4 = 0.250000000000000

Shape functions at xi=1, eta=1:
N1 = 0
N2 = 0
N3 = 1.00000000000000
N4 = 0


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4 & 0 & 0 \\
0 & N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4 & 0\\
0 & 0 & N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4
\end{bmatrix}
$$

In [27]:
def N() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[N1, 0, 0, N2, 0, 0, N3, 0, 0, N4, 0, 0],
     [0, N1, 0, 0, N2, 0, 0, N3, 0, 0, N4],
     [0, 0, N1, 0, 0, N2, 0, 0, N3, 0, 0]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

print("The shape function matrix N is:")
sp.pprint(N())

The shape function matrix N is:
⎡(0.25 - 0.25⋅ξ)⋅(1 - η)             0                        0             (1 ↪
⎢                                                                              ↪
⎢           0             (0.25 - 0.25⋅ξ)⋅(1 - η)             0                ↪
⎢                                                                              ↪
⎣           0                        0             (0.25 - 0.25⋅ξ)⋅(1 - η)     ↪

↪  - η)⋅(0.25⋅ξ + 0.25)             0                        0             (η  ↪
↪                                                                              ↪
↪          0             (1 - η)⋅(0.25⋅ξ + 0.25)             0                 ↪
↪                                                                              ↪
↪          0                        0             (1 - η)⋅(0.25⋅ξ + 0.25)      ↪

↪ + 1)⋅(0.25⋅ξ + 0.25)             0                        0             (0.2 ↪
↪                                                                          

### The shape function matrix with values substituted

In [28]:
# Evaluate the shape function matrix at xi=0.5, eta=1.0
shape_function_matrix = N().subs({XI: 0.5, ETA: 1.0})
print("Shape function matrix N at xi=0.5, eta=1.0:")
sp.pprint(shape_function_matrix)

Shape function matrix N at xi=0.5, eta=1.0:
⎡0  0  0  0  0  0  0.75   0     0    0.25   0     0  ⎤
⎢                                                    ⎥
⎢0  0  0  0  0  0   0    0.75   0     0    0.25   0  ⎥
⎢                                                    ⎥
⎣0  0  0  0  0  0   0     0    0.75   0     0    0.25⎦


### The jacobian matrix

The jacobian matrix does the following:
- maps the natural coordinates (xi, eta) to the physical coordinates (x, y) as it a measure how the physical coordinates change when the natural coordinates change.
- it is used to transform the derivatives of the shape functions from the natural coordinates to the physical coordinates using the chain rule.

So the jacobian is constructed using the chain rule, from the derivatives of the mapping between the global and the local system, and its value depends on the physical coordinates. The Jacobian changes within the element because the mapping from the natural coordinates (xi, eta) to the physical coordinates (x, y) is not necessarily linear. The derivatives of the mapping functions (x_map, y_map) with respect to xi and eta (which form the Jacobian) depend on the shape and distortion of the element in physical space. Since xi and eta vary across the element, the derivatives, and thus the Jacobian, also vary.

$$J = \begin{bmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial y}{\partial \xi} \\
\frac{\partial x}{\partial \eta} & \frac{\partial y}{\partial \eta}
\end{bmatrix}$$

The chain rule for derivation is:

$$\begin{bmatrix}
\frac{\partial}{\partial \xi} \\
\frac{\partial}{\partial \eta}
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial y}{\partial \xi} \\
\frac{\partial x}{\partial \eta} & \frac{\partial y}{\partial \eta}
\end{bmatrix}
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}
= J
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}$$

To express the derivatives with respect to $x$ and $y$ in terms of $\xi$ and $\eta$, you would use the inverse of the Jacobian:

$$
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}
= J^{-1}
\begin{bmatrix}
\frac{\partial}{\partial \xi} \\
\frac{\partial}{\partial \eta}
\end{bmatrix}
$$

$$J^{-1} = \begin{bmatrix}
\frac{\partial \xi}{\partial x} & \frac{\partial \xi}{\partial y} \\
\frac{\partial \eta}{\partial x} & \frac{\partial \eta}{\partial y}
\end{bmatrix}$$

The determinant of the jacobian is used to account for the change of the element area when transforming from the natural to the physical coordinates.

In [29]:
def jacobian() -> np.array:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # Calculate the derivatives
    dx_dxi = sp.diff(x_map, XI)
    dy_dxi = sp.diff(y_map, XI)
    dx_deta = sp.diff(x_map, ETA)
    dy_deta = sp.diff(y_map, ETA)

    return sp.Matrix([[dx_dxi, dy_dxi],
                      [dx_deta, dy_deta]])

# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(coordinate_values)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at some points
for xi_eval in [-0.5773, 0, 0.5773]:
    for eta_eval in [-0.5773, 0, 0.5773]:
        J_evaluated = J_substituted.subs({XI: xi_eval, ETA: eta_eval})
        print(f'Jacobian matrix at xi={xi_eval}, eta={eta_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()


The jacobian matrix with coordinate values substituted is:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦

Jacobian matrix at xi=-0.5773, eta=-0.5773:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=-0.5773, eta=0:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=-0.5773, eta=0.5773:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=0, eta=-0.5773:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=0, eta=0:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=0, eta=0.5773:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=0.5773, eta=-0.5773:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point is |J|=1.00

Jacobian matrix at xi=0.5773, eta=0:
⎡1.0   0 ⎤
⎢        ⎥
⎣ 0   1.0⎦
The determinant at this point 

### Visualizing the determinant of the jacobian

In [30]:
%matplotlib notebook

# Calculate the determinant of the Jacobian
J_det = J.det()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_det_substituted = J_det.subs(coordinate_values)

# Create a function to evaluate the determinant for given xi and eta
J_det_func = sp.lambdify((XI, ETA), J_det_substituted)

# Create a grid of xi and eta values
xi_vals = np.linspace(-1, 1, 50)
eta_vals = np.linspace(-1, 1, 50)
xi_grid, eta_grid = np.meshgrid(xi_vals, eta_vals)

# Evaluate the determinant on the grid
det_values = J_det_func(xi_grid, eta_grid)

if isinstance(det_values, float):
    det_values = np.ones([len(xi_vals), len(eta_vals)]) * det_values

# Map xi and eta values to x and y coordinates
x_vals = sp.lambdify((XI, ETA), x_map.subs(coordinate_values))(xi_grid, eta_grid)
y_vals = sp.lambdify((XI, ETA), y_map.subs(coordinate_values))(xi_grid, eta_grid)

# # move the physical element so the centroid is at 0, 0
x_vals -= np.mean([x1_val, x2_val, x3_val, x4_val])
y_vals -= np.mean([y1_val, y2_val, y3_val, y4_val])

# Create the contour plot
plt.figure()
contour = plt.contourf(x_vals, y_vals, det_values, levels=50, cmap='viridis')

# plot the natural element shape, too
plt.plot([-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1], 'k--', label='Natural Element Shape')

# setting the limits so everything is visible
plt.xlim([1.5 * min(-1, np.min(x_vals)), 1.5 * max(1, np.max(x_vals))])
plt.ylim([1.5 * min(-1, np.min(y_vals)), 1.5 * max(1, np.max(y_vals))])

# adding the real physical coordinates to the positions of the moved nodes
x1_val_pos = x1_val - np.mean([x1_val, x2_val, x3_val, x4_val])
y1_val_pos = y1_val - np.mean([y1_val, y2_val, y3_val, y4_val])
x2_val_pos = x2_val - np.mean([x1_val, x2_val, x3_val, x4_val])
y2_val_pos = y2_val - np.mean([y1_val, y2_val, y3_val, y4_val])
x3_val_pos = x3_val - np.mean([x1_val, x2_val, x3_val, x4_val])
y3_val_pos = y3_val - np.mean([y1_val, y2_val, y3_val, y4_val])
x4_val_pos = x4_val - np.mean([x1_val, x2_val, x3_val, x4_val])
y4_val_pos = y4_val - np.mean([y1_val, y2_val, y3_val, y4_val])

for xpos, ypos, xtext, ytext in zip(
        [x1_val_pos, x2_val_pos, x3_val_pos, x4_val_pos],
        [y1_val_pos, y2_val_pos, y3_val_pos, y4_val_pos],
        [coordinate_values[x1], coordinate_values[x2], coordinate_values[x3], coordinate_values[x4]],
        [coordinate_values[y1], coordinate_values[y2], coordinate_values[y3], coordinate_values[y4]]
):
    plt.plot(xpos, ypos, 'ro')  # plot the physical nodes
    plt.text(xpos, ypos, f'({xtext:.2f}, {ytext:.2f})', fontsize=8, ha='right')

# add a vertical and horizontal line at the origin
plt.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(0, color='k', linestyle='--', linewidth=0.5)

plt.colorbar(contour, label='det(J)')

plt.title('Jacobian of the physical element with the local element.\nCentroids are matched at (0, 0), real coordinates shown.')
plt.xlabel('xi')
plt.ylabel('eta')
plt.axis('equal')
plt.show()

<IPython.core.display.Javascript object>

### Deriving the $B^I$ matrix
This matrix iss associated with the in-plane strains.
$$B^I = \begin{bmatrix}B^I_1 & B^I_2 & B^I_3 & B^I_4\end{bmatrix}$$

where $B^I_j$ is the matrix for node $j$:
$$B^I_j = \begin{bmatrix} 0 & \frac{\partial N_j}{\partial x} & 0 \\ 0 & 0 & -\frac{\partial N_j}{\partial y} \\ 0 & \frac{\partial N_j}{\partial y} & -\frac{\partial N_j}{\partial x} \end{bmatrix}$$

and

$$\frac{\partial N_j}{\partial x} = \frac{\partial N_j}{\partial \xi} \frac{\partial \xi}{\partial x}$$

wich uses the inverse jacobian matrix to transform the derivatives from the natural coordinates to the physical coordinates.

In [31]:
def BI() -> np.array:
    """
    Calculate the BI matrix for the element.
    """
    # the derivatives by the natural coordinates
    dN_dxi, dN_deta = shape_function_derivatives()

    # the inverse jacobian that contains the terms to transform the derivatives. This has 2x2 elements.
    # In row 1 are dxi/dx, dxi/dy
    # In row 2 are deta/dx, deta/dy
    Jinv = J_substituted.inv()  # The inverse of the jacobian

    B_I = None  # Initialize B_I to None

    for node in range(NNODE):
        # Calculate the derivatives by x and y
        dN_dx = dN_dxi[node] * Jinv[0, 0] + dN_deta[node] * Jinv[1, 0]
        dN_dy = dN_dxi[node] * Jinv[0, 1] + dN_deta[node] * Jinv[1, 1]

        # Create the B^I matrix for this node
        # original
        # B_I_node = sp.Matrix([[0, 0, -dN_dx],
        #                      [0, -dN_dy, 0],
        #                      [0, -dN_dx, -dN_dy]])
        # Concepts...
        B_I_node = sp.Matrix([[0, dN_dx, 0],
                             [0, 0, dN_dy],
                             [0, dN_dy, dN_dx]])

        if B_I is None:
            B_I = B_I_node
        else:
            B_I = sp.Matrix.hstack(B_I, B_I_node)

    return B_I

### Deriving the $B^O$ matrix
This matrix iss associated with the off-plane strains.
$$B^O = \begin{bmatrix}B^O_1 & B^O_2 & B^O_3 & B^O_4\end{bmatrix}$$

where $B^O_j$ is the matrix for node $j$:
$$B^O_j = \begin{bmatrix}
\frac{\partial N_j}{\partial x} & 0 & -N_j \\
\frac{\partial N_j}{\partial y} & N_j & 0 \\
\end{bmatrix}$$

In [32]:
def BO() -> np.array:
    """
    Calculate the BO matrix for the element.
    """
    # the shape functions
    N = shape_functions()

    # the derivatives by the natural coordinates
    dN_dxi, dN_deta = shape_function_derivatives()

    # the inverse jacobian that contains the terms to transform the derivatives. This has 2x2 elements.
    # In row 1 are dxi/dx, dxi/dy
    # In row 2 are deta/dx, deta/dy
    Jinv = J_substituted.inv()  # The inverse of the jacobian

    B_O = None  # Initialize B_O to None

    for node in range(NNODE):
        # Calculate the derivatives by x and y
        dN_dx = dN_dxi[node] * Jinv[0, 0] + dN_deta[node] * Jinv[1, 0]
        dN_dy = dN_dxi[node] * Jinv[0, 1] + dN_deta[node] * Jinv[1, 1]

        # # Create the B^O matrix for this node
        # original
        # B_O_node = sp.Matrix([[dN_dx, -N[node], 0],
        #                       [dN_dy, 0, -N[node]]])
        # concepts...
        B_O_node = sp.Matrix([[-dN_dy, 0, N[node]],
                              [-dN_dx, N[node], 0]])

        if B_O is None:
            B_O = B_O_node
        else:
            B_O = sp.Matrix.hstack(B_O, B_O_node)

    return B_O

### Let's see a real world example
The plate is simply the same as the one in the natural coordinates

In [40]:
bi_matrix = BI()
bo_matrix = BO()

# Substitute the symbolic coordinates with numerical values
bi_matrix_evaluated = bi_matrix.subs({XI: -0.5773, ETA: -0.5773})
bo_matrix_evaluated = bo_matrix.subs({XI: -0.5773, ETA: -0.5773})

print("BI Matrix at xi=0, eta=0 with substituted coordinates:")
sp.pprint(bi_matrix_evaluated)
print()
print("BO Matrix at xi=0, eta=0 with substituted coordinates:")
sp.pprint(bo_matrix_evaluated)

BI Matrix at xi=0, eta=0 with substituted coordinates:
⎡0  -0.394325      0      0  0.394325       0      0  0.105675     0      0  - ↪
⎢                                                                              ↪
⎢0      0      -0.394325  0      0      -0.105675  0     0      0.105675  0    ↪
⎢                                                                              ↪
⎣0  -0.394325  -0.394325  0  -0.105675  0.394325   0  0.105675  0.105675  0  0 ↪

↪ 0.105675      0    ⎤
↪                    ⎥
↪    0      0.394325 ⎥
↪                    ⎥
↪ .394325   -0.105675⎦

BO Matrix at xi=0, eta=0 with substituted coordinates:
⎡0.394325       0        0.6219688225  0.105675        0        0.1666811775   ↪
⎢                                                                              ↪
⎣0.394325  0.6219688225       0        -0.394325  0.1666811775       0         ↪

↪ -0.105675       0        0.0446688225  -0.394325       0        0.1666811775 ↪
↪                                          

### C matrix - compliance matrix

This is given for plane stress and plane strain problems in Eqs. 2.31, 2.32.

The compliance matrix c for the out-of-plane (bending) part needs to relate moments to curvatures. For an isotropic Reissner-Mindlin plate, this relationship can be expressed in terms of the material properties $ E $ (Young's modulus) and $ \nu $ (Poisson's ratio). The 2x2 compliance matrix $ C_b $ relates the bending moments to the curvatures and is given by:

$$C_b = \begin{bmatrix} D & D\nu \\ D\nu & D \end{bmatrix}$$

where $ D $ is the flexural rigidity of the plate, defined as:

$$D = \frac{E t^3}{12(1 - \nu^2)}$$

Here, $ t $ is the thickness of the plate.

In [34]:
# Flexural rigidity, without thickness
D = E / (1 - nu**2)
# Bending material constitutive matrix
c_b = D * sp.Matrix([[1, nu, 0],
                     [nu, 1, 0],
                     [0, 0, (1 - nu) / 2]])

print("Bending material matrix C_b:")
sp.pprint(c_b)

# Shear modulus
G = E / (2 * (1 + nu))
# Shear material constitutive matrix, without thickness
c_s = G * sp.Matrix([[1, 0],
                     [0, 1]])

print("\nShear material matrix C_s:")
sp.pprint(c_s)

Bending material matrix C_b:
⎡213333333333.333  53333333333.3333        0      ⎤
⎢                                                 ⎥
⎢53333333333.3333  213333333333.333        0      ⎥
⎢                                                 ⎥
⎣       0                 0          80000000000.0⎦

Shear material matrix C_s:
⎡80000000000.0        0      ⎤
⎢                            ⎥
⎣      0        80000000000.0⎦


## Getting the stiffness matrix
This is done by numerically integtrating the stiffness matrix over the element area using the Gauss quadrature method.

### Quadrature points and weights

In [35]:
gi_data = {
    1: {
        'point': [0],
        'weight': [2]
    },
    2: {
        'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
        'weight': [1, 1]
    },
    3: {
        'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
        'weight': [5 / 9,  8 / 9, 5 / 9]
    },
    4: {
        'point': [-0.861136, -0.339981, 0.339981, 0.861136],
        'weight': [0.347855, 0.652145, 0.652145, 0.347855]
    },
    5: {
        'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
        'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
    },
    6: {
        'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
        'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
    }
}

In [36]:
### Helper function for displaying the eigenmodes of the stiffness matrix

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

def plot_eigenshape(ax, eigenvector, coords):
    """
    Plots a 3D eigenshape for the 4-node plate element.

    :param ax: The matplotlib 3D axis object to plot on.
    :param eigenvector: The eigenvector representing the mode shape (12x1 array).
    :param coords: A dictionary with the physical coordinates of the nodes.
    """
    # 1. Extract the physical coordinates of the nodes
    x_coords = np.array([coords[x1], coords[x2], coords[x3], coords[x4]])
    y_coords = np.array([coords[y1], coords[y2], coords[y3], coords[y4]])
    z_coords = np.zeros(4)  # Initial z-coordinates are 0

    # 2. Draw the original, undeformed element
    verts_orig = [list(zip(x_coords, y_coords, z_coords))]
    poly_orig = Poly3DCollection(verts_orig, alpha=0.2, facecolors='cyan', edgecolors='k')
    ax.add_collection3d(poly_orig)

    # 3. Calculate the deformed shape with automatic scaling
    # Extract the w-displacements (vertical) from the eigenvector
    w_displacements_raw = np.real_if_close(eigenvector[0::3])

    # Calculate a characteristic length of the element (diagonal of the bounding box)
    char_length = np.sqrt((np.max(x_coords) - np.min(x_coords))**2 + (np.max(y_coords) - np.min(y_coords))**2)

    # Find the maximum absolute displacement in the mode
    max_disp = np.max(np.abs(w_displacements_raw))

    # Calculate the scaling factor to make the displacement visible
    scale_factor = 1.0
    if max_disp > 1e-9:  # Prevent division by zero
        desired_scale_fraction = 0.25
        scale_factor = (desired_scale_fraction * char_length) / max_disp

    w_displacements = w_displacements_raw * scale_factor

    # 4. Create a grid in the natural coordinate system for a smooth surface
    xi_vals = np.linspace(-1, 1, 20)
    eta_vals = np.linspace(-1, 1, 20)
    xi_grid, eta_grid = np.meshgrid(xi_vals, eta_vals)

    # Lambdify Shape- and Mapping-functions for fast evaluation
    N_funcs = [sp.lambdify((XI, ETA), n) for n in shape_functions()]
    x_map_func = sp.lambdify((XI, ETA), x_map.subs(coordinate_values))
    y_map_func = sp.lambdify((XI, ETA), y_map.subs(coordinate_values))

    # Calculate the physical (x, y) and deformed (z) coordinates on the grid
    x_surf = x_map_func(xi_grid, eta_grid)
    y_surf = y_map_func(xi_grid, eta_grid)
    z_surf = np.zeros_like(xi_grid)
    for i in range(NNODE):
        z_surf += N_funcs[i](xi_grid, eta_grid) * w_displacements[i]

    # 5. Plot the deformed surface, colored by displacement
    norm = plt.Normalize(z_surf.min(), z_surf.max())
    colors = plt.cm.viridis(norm(z_surf))
    ax.plot_surface(x_surf, y_surf, z_surf, facecolors=colors, alpha=0.8, shade=True)

    # 6. Set axis labels and title
    ax.set_xlabel('X-Coordinate')
    ax.set_ylabel('Y-Coordinate')
    ax.set_zlabel('W-Displacement')
    ax.set_title('Eigenshape')
    ax.axis('equal')
    ax.view_init(elev=20, azim=-30)  # Adjust view angle for better visibility


 ### Getting the stiffness matrix

The membrane part of the integral in (8.20) is carried out here first.

In [37]:
%matplotlib notebook

NG = 2  # number of gauss points in each direction

kappa = 5 / 6  # shear correction factor

def get_KI():
    """
    Calculates the in-plane stiffness matrix K_I using Gauss quadrature.
    """
    # Initialize the stiffness matrix as a zero matrix
    K_I = sp.zeros(ND * NNODE, ND * NNODE)

    # Calculate the symbolic B^I matrix once, before the loop
    B_I_symbolic = BI()


    # Lambdify the jacobian determinant for faster evaluation
    det_J_func = sp.lambdify((XI, ETA), J_substituted.det(), 'numpy')

    for i in range(NG):
        for j in range(NG):
            xi = gi_data[NG]['point'][i]
            eta = gi_data[NG]['point'][j]
            weight = gi_data[NG]['weight'][i] * gi_data[NG]['weight'][j]

            # Evaluate the determinant of the jacobian at the gauss point
            det_J_gauss = det_J_func(xi, eta)

            # Substitute the gauss point coordinates into the symbolic B^I matrix
            B_I_gauss = B_I_symbolic.subs({XI: xi, ETA: eta})

            # Calculate the stiffness matrix contribution
            K_I_contribution = weight * det_J_gauss * (t ** 3 / 12) * (B_I_gauss.T * c_b * B_I_gauss)

            K_I += K_I_contribution

    return K_I


def get_KI_fast():
    """
    Calculates the in-plane stiffness matrix K_I using Gauss quadrature
    with lambdified functions for speed.
    """
    K_I = np.zeros((ND * NNODE, ND * NNODE))

    # 1. Symbolische Matrizen einmalig erstellen
    B_I_symbolic = BI()
    det_J_symbolic = J_substituted.det()

    # 2. Lambdify: Symbolische Ausdrücke in schnelle numerische Funktionen umwandeln
    #    Die resultierenden Funktionen erwarten xi und eta als Argumente.
    B_I_func = sp.lambdify((XI, ETA), B_I_symbolic, 'numpy')
    det_J_func = sp.lambdify((XI, ETA), det_J_symbolic, 'numpy')

    # Konvertieren Sie die symbolische c_b-Matrix in eine numerische NumPy-Array
    c_b_numeric = np.array(c_b.evalf()).astype(float)

    # 3. Numerische Integrationsschleife
    for i in range(NG):
        for j in range(NG):
            xi = gi_data[NG]['point'][i]
            eta = gi_data[NG]['point'][j]
            weight = gi_data[NG]['weight'][i] * gi_data[NG]['weight'][j]

            # Schnelle numerische Auswertung an den Gausspunkten
            det_J_gauss = det_J_func(xi, eta)
            B_I_gauss = B_I_func(xi, eta)

            # Numerische Matrixmultiplikation mit NumPy
            K_I_contribution = weight * det_J_gauss * (t ** 3 / 12) * (B_I_gauss.T @ c_b_numeric @ B_I_gauss)
            K_I += K_I_contribution

    return K_I

def get_KO(NG_local=2):

    """
    Calculates the in-plane stiffness matrix K_I using Gauss quadrature.
    """
    # Initialize the stiffness matrix as a zero matrix
    K_O = sp.zeros(ND * NNODE, ND * NNODE)

    # Calculate the symbolic B^I matrix once, before the loop
    B_O_symbolic = BO()

    # Lambdify the jacobian determinant for faster evaluation
    det_J_func = sp.lambdify((XI, ETA), J_substituted.det(), 'numpy')

    for i in range(NG_local):
        for j in range(NG_local):
            xi = gi_data[NG_local]['point'][i]
            eta = gi_data[NG_local]['point'][j]
            weight = gi_data[NG_local]['weight'][i] * gi_data[NG_local]['weight'][j]

            # Evaluate the determinant of the jacobian at the gauss point
            det_J_gauss = det_J_func(xi, eta)

            # Substitute the gauss point coordinates into the symbolic B^O matrix
            B_O_gauss = B_O_symbolic.subs({XI: xi, ETA: eta})

            # Calculate the stiffness matrix contribution
            K_O_contribution = weight * det_J_gauss * t * kappa * (B_O_gauss.T * c_s * B_O_gauss)

            K_O += K_O_contribution

    return K_O

def get_KO_fast(NG_local=2):
    """
    Calculates the out-of-plane (shear) stiffness matrix K_O using Gauss quadrature
    with lambdified functions for speed.
    """
    K_O = np.zeros((ND * NNODE, ND * NNODE))
    kappa = 5 / 6  # shear correction factor

    # 1. Symbolische Matrizen einmalig erstellen
    B_O_symbolic = BO()
    det_J_symbolic = J_substituted.det()

    # 2. Lambdify: Symbolische Ausdrücke in schnelle numerische Funktionen umwandeln
    B_O_func = sp.lambdify((XI, ETA), B_O_symbolic, 'numpy')
    det_J_func = sp.lambdify((XI, ETA), det_J_symbolic, 'numpy')

    # Konvertieren Sie die symbolische c_s-Matrix in eine numerische NumPy-Array
    c_s_numeric = np.array(c_s.evalf()).astype(float)

    # 3. Numerische Integrationsschleife
    for i in range(NG_local):
        for j in range(NG_local):
            xi = gi_data[NG_local]['point'][i]
            eta = gi_data[NG_local]['point'][j]
            weight = gi_data[NG_local]['weight'][i] * gi_data[NG_local]['weight'][j]

            # Schnelle numerische Auswertung an den Gausspunkten
            det_J_gauss = det_J_func(xi, eta)
            B_O_gauss = B_O_func(xi, eta)

            # Numerische Matrixmultiplikation mit NumPy
            K_O_contribution = weight * det_J_gauss * t * kappa * (B_O_gauss.T @ c_s_numeric @ B_O_gauss)
            K_O += K_O_contribution

    return K_O


# K = get_KI()
# K = get_KO()
K = get_KI_fast() + get_KO_fast()
# K = np.array(K.evalf()).astype(float)

eigvals, shapes = np.linalg.eig(K)

# Sort the eigenvalues and corresponding eigenvectors
sorted_indices = np.argsort(eigvals)
sorted_eigvals = np.real(eigvals[sorted_indices])
sorted_shapes = np.real(shapes[:, sorted_indices])

for i in range(sorted_shapes.shape[1]):
    shape = sorted_shapes[:, i]
    print()
    print(f"--- Form {i} (Eigenwert: {sorted_eigvals[i]:.2e}) ---")
    # Drucken Sie den Eigenvektor mit einer geeigneten Formatierung
    print(shape)


print(sorted_eigvals)

for i in range(12):
    print(np.linalg.norm(sorted_shapes[:, i]))
print()

# Plotten Sie die ersten 6 Eigenformen
fig, axes = plt.subplots(2, 3, figsize=(9, 6), subplot_kw={'projection': '3d'})
axes = axes.flatten()

for i in range(6):
    ax = axes[i]
    shape = sorted_shapes[:, i]
    plot_eigenshape(ax, shape, coordinate_values)
    ax.set_title(f'Mode {i}\nEigenvalue: {sorted_eigvals[i]:.2e}')

plt.tight_layout()
plt.show()

# Plotten Sie die ersten 6 Eigenformen
fig, axes = plt.subplots(2, 3, figsize=(9, 6), subplot_kw={'projection': '3d'})
axes = axes.flatten()

for i in range(6, 12):
    ax = axes[i-6]
    shape = sorted_shapes[:, i]
    plot_eigenshape(ax, shape, coordinate_values)
    ax.set_title(f'Mode {i}\nEigenvalue: {sorted_eigvals[i]:.2e}')

plt.tight_layout()
plt.show()





--- Form 0 (Eigenwert: -9.54e-07) ---
[-7.07106781e-01  1.76776695e-01  1.76776695e-01 -3.53553391e-01
  1.76776695e-01  1.76776695e-01 -2.82815916e-16  1.76776695e-01
  1.76776695e-01 -3.53553391e-01  1.76776695e-01  1.76776695e-01]

--- Form 1 (Eigenwert: 4.04e-06) ---
[ 0.27761824 -0.11491215 -0.25204777  0.04779393 -0.11491215 -0.25204777
 -0.4563016  -0.11491215 -0.25204777 -0.22647729 -0.11491215 -0.25204777]

--- Form 2 (Eigenwert: 4.04e-06) ---
[ 0.27761824 -0.11491215 -0.25204777  0.04779393 -0.11491215 -0.25204777
 -0.4563016  -0.11491215 -0.25204777 -0.22647729 -0.11491215 -0.25204777]

--- Form 3 (Eigenwert: 8.26e+09) ---
[ 0.32786659 -0.26693024 -0.26693024 -0.32786659 -0.26693024  0.26693024
  0.32786659  0.26693024  0.26693024 -0.32786659  0.26693024 -0.26693024]

--- Form 4 (Eigenwert: 1.56e+10) ---
[-2.22398875e-16  4.98823394e-01 -3.42815119e-02  1.27916119e-16
 -4.98823394e-01  3.42815119e-02 -2.42238951e-17  4.98823394e-01
 -3.42815119e-02  2.12810625e-16 -4.988233

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### exporting the code

In [38]:


from sympy.utilities.codegen import codegen
